In [ ]:
from sklearn.model_selection import KFold
import pandas as pd

df = pd.read_csv('/Users/markun/git/rsna2024/kaggle/data/train.csv')
df

In [ ]:
"""
fold: 0
train_study_id: 
  - 4003253
  - 7143189
valid_study_id:
  - 4646740
  - 8785691
"""

import yaml


kf = KFold(n_splits=5, shuffle=True, random_state=42)
for fold, (trn_idx, val_idx) in enumerate(kf.split(df)):

    print('#'*30)
    print(f'start fold {fold}')
    print('#'*30)
    print(len(trn_idx), len(val_idx))

    df_train = df.iloc[trn_idx]
    df_valid = df.iloc[val_idx]
    
    train = df_train['study_id'].tolist() 
    valid = df_valid['study_id'].tolist()  

    fold_info = {
        'fold': fold,
        'train_study_id': train,
        'valid_study_id': valid
    }

    output_yaml = f'fold_{fold}.yaml'
    with open(output_yaml, 'w') as yaml_file:
        yaml.dump(fold_info, yaml_file, default_flow_style=False)

    print(f"YAMLファイル '{output_yaml}' に書き出しました。")

In [ ]:
import timm

timm.list_models()